# Homework 5 (80 points)

In this homework, you will learn about:

* Design and implementation of a Graph library
* Representation of graphs as adjacency lists
* Breadth-First Search
* Depth-First Search
* Topological Sort

**Note that I made the deliberate choice of giving you very little instructions about how to design your graph library API. This is on purpose: one of the most important skill for a software engineer is API design. However, note that I will discuss all of this in class, so if you are confused about what you have to do, make sure to tune in. Also note that if you are thoughtful about your design, you will have to write much less code.**

In [87]:
from math import inf

You will use the following class to represent vertices in your graph. A vertex has an identifier, and it can also have a set of attributes. An identifier is almost like an attribute, except that it needs to be provided to create a vertex.

In [119]:
class Vertex:
    
    def __init__(self,id):
        self.attributes = {}
        self.attributes['id'] = id
        
    def __str__(self):
        return str(self.attributes)
        
    def new_copy(self):
        return Vertex(self.attributes['id'])
        
    def set(self,key,value):
        self.attributes[key] = value
        
    def get(self,key):
        return self.attributes[key]

In [120]:
class Graph:
    def __init__(self):
        self.vertices = []
        self.id_to_v = {}
        self.v_to_v = {}
        
    def add_vertex(self, v):
        self.vertices.append(v)
        self.id_to_v[v.get('id')] = v
        
    def add_edge(self, v1, v2):
        pass
    
    def BFS(self, v):
        Q = [v.get('id')]
        result = []
        while len(Q) > 0:
            target = Q.pop(0)
            result.append(target)
            for adj in self.v_to_v[target]:
                if adj not in result and adj not in Q:
                    Q.append(adj)
        return result
    
    def DFS(self):
        self.stack = []
        self.isAcyclic = True
        for v in self.vertices:
            v.set('color', 0)
        self.time = 0
        for v in self.vertices:
            if v.get('color') == 0:
                self.DFS_visit(v)
    
    def DFS_visit(self, v):
        v.set('color', 1)
        self.time += 1
        v.set('d', self.time)
        adjs = self.v_to_v.get(v.get('id'))
        if adjs:
            for u_id in adjs:
                u = self.id_to_v[u_id]
                if u.get('color') == 0:
                    self.DFS_visit(u)
                elif u.get('color') == 1:
                    self.isAcyclic = False
        v.set('color', 2)
        self.time += 1
        v.set('f', self.time)
        self.stack.append(v.get('id'))

Your library will provide undirected graphs. Note that you may not change the name of the class, but you can change its inheritance, and you will need to add methods. 

In [122]:
class UndirectedGraph(Graph):
    def add_edge(self, v1, v2):
        v1_id = v1.get('id')
        v2_id = v2.get('id')
        if v1_id not in self.v_to_v:
            self.v_to_v[v1_id] = set()
        self.v_to_v[v1_id].add(v2_id)
        
        if v2_id not in self.v_to_v:
            self.v_to_v[v2_id] = set()
        self.v_to_v[v2_id].add(v1_id)

Likewise, your library will provide directed graphs.

In [123]:
class DirectedGraph(Graph):
    def add_edge(self, v1, v2):
        v1_id = v1.get('id')
        v2_id = v2.get('id')
        if v1_id not in self.v_to_v:
            self.v_to_v[v1_id] = set()
        self.v_to_v[v1_id].add(v2_id)
    
    # acyclic graph : directed graph with no directed cycles
    def acyclic(self):
        self.DFS()
        return self.isAcyclic
    
    def topological_sort(self):
        self.DFS()
        return self.stack[::-1]

Both kinds of graphs should have an API that allows us to create new graphs. First, there should be a method 'add_vertex' that takes a vertex as an input and adds it to the graph. Second, there should be a method 'add_edge' that takes two vertices and adds an edge between them in the graph. For example, your implementation should work with the following codes.

In [124]:
def create_graph_1():
    G = UndirectedGraph()

    for i in ['r','s','t','u','v','w','x','y']:
        G.add_vertex(Vertex(i))
    
    for (v1,v2) in [('v','r'),('r','s'),('s','w'),
              ('w','t'),('w','x'),('t','x'),
              ('t','u'),('x','u'),('x','y'),('u','y')]:
        G.add_edge(G.id_to_v[v1],G.id_to_v[v2])
    
    return G

In [125]:
G1 = create_graph_1()

In [126]:
def create_graph_2():
    G = DirectedGraph()
    
    for i in ['u','v','w','x','y','z']:
        G.add_vertex(Vertex(i))
        
    for (v1,v2) in [('u','x'),('u','v'),('x','v'),('v','y'),
                    ('y','x'),('w','y'),('w','z'),('z','z')]:
        
        G.add_edge(G.id_to_v[v1],G.id_to_v[v2])
        
    return G

In [127]:
G2 = create_graph_2()

In [128]:
def create_graph_3():
    G = DirectedGraph()
    
    for i in ['u','v','w','x','y','z']:
        G.add_vertex(Vertex(i))
        
    for (v1,v2) in [('u','x'),('u','v'),('v','y'),
                    ('y','x'),('w','y'),('w','z')]:
        
        G.add_edge(G.id_to_v[v1],G.id_to_v[v2])
        
    return G    

In [129]:
G3 = create_graph_3()

Please note two important things. First, the methods add_vertex and add_edge do not return a new graph, instead they update the graph in-place. Second, in the graph implementation, you need to work with vertex objects, and not their identifier. It means that to interface with the graph, the graph needs to provide a dictionnary 'id_to_v' that keeps track of which identifier corresponds to which vertex.

**Problem 1: (20 points)** Implement Breads-First Search (BFS) for both kinds of graphs. The BFS method takes a vertex as an argument and should return a list of identifiers. Note that there can be more than one right answer. *The following call should work.*

In [130]:
G1.BFS(G1.id_to_v['s'])
#answer : ['s', 'r', 'w', 'v', 't', 'x', 'u', 'y']

['s', 'w', 'r', 'x', 't', 'v', 'u', 'y']

**Problem 2: (20 points)** Implement Depth-First Search (BFS) for both kinds of graphs. The DFS method takes no arguments and does not return anything. Instead, the DFS method should timestamp the vertices. *The following call should work.*

In [131]:
G2.DFS()

In [132]:
for v in G2.vertices:
    print(v)

{'id': 'u', 'color': 2, 'd': 1, 'f': 8}
{'id': 'v', 'color': 2, 'd': 3, 'f': 6}
{'id': 'w', 'color': 2, 'd': 9, 'f': 12}
{'id': 'x', 'color': 2, 'd': 2, 'f': 7}
{'id': 'y', 'color': 2, 'd': 4, 'f': 5}
{'id': 'z', 'color': 2, 'd': 10, 'f': 11}


**Problem 3: (20 points)** Implement a method to test if a directed graph is acyclic. The methods takes no argument but returns a boolean. *The following call should work.*

In [133]:
G3.acyclic()
# answer : True 

True

**Problem 4: (20 points)** Implement a method that computes a topological sort for a directed graph. The method takes no arguments and returns a list of identifiers. *The following call should work.*

In [134]:
G3.topological_sort()
#answer : ['w', 'z', 'u', 'v', 'y', 'x']

['w', 'z', 'u', 'v', 'y', 'x']